In [ ]:
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore') 
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)

     |████████████████████████████████| 6.3MB 4.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=0ce4fbd45411eeaba01a336995f2cf5c3b071cb0860fea6c4b5f8f9e1b23a5a4
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
nse=pd.read_csv('/content/gdrive/MyDrive/SASH/Nifty50.csv')
nse['Symbol']=nse['Symbol'] + '.NS'

# CODE CREATION

In [ ]:
df1 = yf.download('HDFC.NS', 
                  start='2021-05-05', 
                  end='2021-06-26',
                  interval='15m')
df1.reset_index(inplace=True)
df1=df1.dropna().reset_index(drop=True)
df1['Datetime']=df1['Datetime'].astype('str')
# new
olddf=pd.DataFrame(columns=['Datetime','Open','High','Low','Close','Checking High','Checking Low'])
# take 25 candles 1day 15min chart as 1 dataframe
for i in range(0,len(df1),25):
  newdf=df1[i:i+25:]
  # For high
  Checking_High=[]
  t=0
  temporary_high=newdf['High'][i] # temperory high will be 1st 15min candle's high
  for j in range(i,i+25):
    if newdf['High'][j] > temporary_high:
      t=t+1  # t will get update with every new high value found
      if t==1:
        temporary_high= newdf['High'][j] # once new high found tempory high will be updated. only once as we dont want candle high to be broken
      Checking_High.append(t)
    else:
      Checking_High.append(0)
  # For low, similar vice versa logic for low
  Checking_Low=[]
  m=0
  temporary_low=newdf['Low'][i]
  for k in range(i,i+25):
    if newdf['Low'][k] < temporary_low:
      m=m+1
      if m==1:
        temporary_low=newdf['Low'][k]
      Checking_Low.append(m)
    else:
      Checking_Low.append(0)
  newdf['Checking High']=Checking_High
  newdf['Checking Low']=Checking_Low
  olddf=olddf.append(newdf,ignore_index=True)
# Now we need to take only days where max high count went upto 1 means only 1 high and 1 low got created.
finaldf=pd.DataFrame(columns=['Datetime','Open','High','Low','Close','Checking High','Checking Low'])
for i in range(0,len(df1),25):
  newdf=olddf[i:i+25:]
  if (max(newdf['Checking High']) == 1 and max(newdf['Checking Low']) == 1): # only 1 time selection
    finaldf=finaldf.append(newdf,ignore_index=True)
finaldf=finaldf[finaldf['Datetime'].str.contains('09:15')].reset_index(drop=True) # remove rest of the data.
finaldf=finaldf.drop_duplicates().reset_index(drop=True)
finaldf=finaldf[['Datetime']] # take only needed one
finaldf['Datetime']=finaldf['Datetime'].astype('str')
finaldf=finaldf.rename(columns={'Datetime':'Date'})
finaldf['Date']=finaldf['Date'].str.split(' ',expand=True)[0] # final date selection

[*********************100%***********************]  1 of 1 completed


In [ ]:
finaldf

,Date
0,2021-05-05


# FOR BACKTEST

In [ ]:
def checker(stock_name,start_date,end_date):
  #**Condition 1**
  #1st 15min high low should get break. That means days's high and day's low should be greater and lesser than 15min 1st candle high and low.
  df1 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='15m')
  df1.reset_index(inplace=True)
  df1=df1.dropna().reset_index(drop=True)
  df1['Datetime']=df1['Datetime'].astype('str')
  # new
  olddf=pd.DataFrame(columns=['Datetime','Open','High','Low','Close','Checking High','Checking Low'])
  # take 25 candles 1day 15min chart as 1 dataframe
  for i in range(0,len(df1),25):
    newdf=df1[i:i+25:]
    # For high
    Checking_High=[]
    t=0
    temporary_high=newdf['High'][i] # temperory high will be 1st 15min candle's high
    for j in range(i,i+25):
      if newdf['High'][j] > temporary_high:
        t=t+1  # t will get update with every new high value found
        if t==1:
          temporary_high= newdf['High'][j] # once new high found tempory high will be updated. only once as we dont want candle high to be broken
        Checking_High.append(t)
      else:
        Checking_High.append(0)
    # For low, similar vice versa logic for low
    Checking_Low=[]
    m=0
    temporary_low=newdf['Low'][i]
    for k in range(i,i+25):
      if newdf['Low'][k] < temporary_low:
        m=m+1
        if m==1:
          temporary_low=newdf['Low'][k]
        Checking_Low.append(m)
      else:
        Checking_Low.append(0)
    newdf['Checking High']=Checking_High
    newdf['Checking Low']=Checking_Low
    olddf=olddf.append(newdf,ignore_index=True)
  # Now we need to take only days where max high count went upto 1 means only 1 high and 1 low got created.
  finaldf=pd.DataFrame(columns=['Datetime','Open','High','Low','Close','Checking High','Checking Low'])
  for i in range(0,len(df1),25):
    newdf=olddf[i:i+25:]
    if (max(newdf['Checking High']) == 1 and max(newdf['Checking Low']) == 1): # only 1 time selection
      finaldf=finaldf.append(newdf,ignore_index=True)
  finaldf=finaldf[finaldf['Datetime'].str.contains('09:15')].reset_index(drop=True) # remove rest of the data.
  finaldf=finaldf.drop_duplicates().reset_index(drop=True)
  finaldf=finaldf[['Datetime','Close']] # take only needed one
  finaldf['Datetime']=finaldf['Datetime'].astype('str')
  finaldf=finaldf.rename(columns={'Datetime':'Date'})
  try:
    finaldf['Date']=finaldf['Date'].str.split(' ',expand=True)[0] # final date selection
  except:
    pass
  return list(finaldf['Close']),list(finaldf['Date'])

In [ ]:
%%capture
# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nse['Symbol']):
    selected_stocks[i]=checker(i,'2021-05-01','2021-05-31')
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)

# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')
final['Stock']='NSE:' + final['Stock'].str.replace('.NS',',')

In [ ]:
%cd '/content/gdrive/MyDrive/SASH'

/content/gdrive/MyDrive/SASH


In [ ]:
final.to_excel('May_BTST_STBT.xlsx',index=False)
files.download('May_BTST_STBT.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>